In [2]:
import pandas as pd
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

In [3]:
df = pd.read_excel("Product review.xlsx")
df = df[['Review']]
df.columns = ['review']
sample_reviews = str(df['review'][10:20])

In [7]:
prompt = '''
Tasks:
generate the summay of below reviews.
mention how many are positive, negative and neutral reviews along with its top 5 key words.
menation the top 5 topic all reviews are talking about

Instruction:
Give me output in python dictionary like this

Answer : {'summary':'[output]' , 'positive_review_count': "[output]", "top 5 positive key words":"[output]",
 'negative_review_count': "[output]", "top 5 negative key words":"[output]",
 "top 5 topic": "[output]"}
 Only give me above said answer dictionary
'''

In [8]:
final_prompt = prompt + ":\n" + sample_reviews

In [10]:
! pip install clarifai-grpc


  Obtaining dependency information for clarifai-grpc from https://files.pythonhosted.org/packages/96/4d/991d28419e3f31dcf14e5b5801dea471928ad97e9fc7815836dbf832f249/clarifai_grpc-10.0.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/224.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/224.8 kB ? eta -:--:--
   --------------------- ------------------ 122.9/224.8 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 224.8/224.8 kB 2.8 MB/s eta 0:00:00


In [15]:

######################################################################################################
# In this section, we set the user authentication, user and app ID, model details, and the URL of
# the text we want as an input. Change these strings to run your own example.
######################################################################################################

# Your PAT (Personal Access Token) can be found in the portal under Authentification
PAT = 'd264c5f1a9b047419752c7e50d8ec553'
# Specify the correct user_id/app_id pairings
# Since you're making inferences outside your app's scope
USER_ID = 'meta'
APP_ID = 'Llama-2'
# Change these to whatever model and text URL you want to use
MODEL_ID = 'llama2-70b-chat'
MODEL_VERSION_ID = 'acba9c1995f8462390d7cb77d482810b'
RAW_TEXT = prompt
# TEXT_FILE_URL = 'https://samples.clarifai.com/negative_sentence_12.txt'
# Or, to use a local text file, assign the url variable
# TEXT_FILE_LOCATION = 'YOUR_TEXT_FILE_LOCATION_HERE'

############################################################################
# YOU DO NOT NEED TO CHANGE ANYTHING BELOW THIS LINE TO RUN THIS EXAMPLE
############################################################################



channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

# To use a local text file, uncomment the following lines
# with open(TEXT_FILE_LOCATION, "rb") as f:
#    file_bytes = f.read()

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    text=resources_pb2.Text(
                        raw=RAW_TEXT
                        # url=TEXT_FILE_URL
                        # raw=file_bytes
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception(f"Post model outputs failed, status: {post_model_outputs_response.status.description}")

# Since we have one input, one output will exist here
output = post_model_outputs_response.outputs[0]

print("Completion:\n")
print(output.data.text.raw)

Completion:


Reviews:

1. The hotel was super clean and comfortable! The staff was friendly and helpful. The location was perfect for exploring the city.
2. The room was small but clean and comfortable. The bathroom was a bit dated but worked well. The staff was friendly.
3. The hotel was nice but the bed was a bit too soft for my liking. The location was great and the staff was friendly.
4. The hotel was not worth the money. The room was small and dirty. The staff was not friendly.
5. The hotel was great! The room was clean and comfortable. The staff was friendly and helpful. The location was perfect.
6. The hotel was okay. The room was clean but the bed was uncomfortable. The staff was friendly.
7. The hotel was not good. The room was dirty and the staff was not friendly.
8. The hotel was great! The room was clean and comfortable. The staff was friendly and helpful. The location was perfect.
9. The hotel was not worth the money. The room was small and dirty. The staff was not friend

In [16]:
# Access the 'data' field and extract the 'text' field
data = output.data
text = data.text

# Access the 'raw' field inside 'text' to get the 'summary' dictionary
summary_dictionary = text.raw

In [17]:
start_index = summary_dictionary.find("{")  # Find the index of '{'
end_index = summary_dictionary.rfind("}")  # Find the index of '}' from the end

if start_index != -1 and end_index != -1:
    summary_dict_str = summary_dictionary[start_index:end_index + 1]  # Extract the substring containing the dictionary
    print(summary_dict_str)

{
'summary': 'The hotel is clean and comfortable with friendly staff. The location is perfect for exploring the city. Some guests found the bed to be too soft or the room to be small, but overall it is a great value for the price.',
'positive_review_count': 6,
'top 5 positive key words': ['clean', 'comfortable', 'friendly', 'perfect', 'great'],
'negative_review_count': 4,
'top 5 negative key words': ['small', 'dirty', 'uncomfortable', 'not worth', 'not friendly'],
'top 5 topic': ['cleanliness', 'comfort', 'staff', 'location', 'value']
}


In [20]:
import ast
summary_dict = ast.literal_eval(summary_dict_str)

print(summary_dict)

{'summary': 'The hotel is clean and comfortable with friendly staff. The location is perfect for exploring the city. Some guests found the bed to be too soft or the room to be small, but overall it is a great value for the price.', 'positive_review_count': 6, 'top 5 positive key words': ['clean', 'comfortable', 'friendly', 'perfect', 'great'], 'negative_review_count': 4, 'top 5 negative key words': ['small', 'dirty', 'uncomfortable', 'not worth', 'not friendly'], 'top 5 topic': ['cleanliness', 'comfort', 'staff', 'location', 'value']}


In [21]:
type(summary_dict)

str

In [23]:
type(summary_dict)

dict